In [13]:
# IMPORT PACKAGES
import arcpy
from arcpy import metadata as md

import os
import pandas as pd

import logging
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

# ABOUT THE TARGET GEODATABASE
sr = arcpy.SpatialReference(3857)

fldr = r"C:\Users\Laura\Documents\Keepsakes\Travel\0_MetadataInstructions\2022 Database Migration"
name = r"Travel_Archive"
gdb = name + ".gdb"

# READ IN THE TEMPLATE SPREADSHEET
xlsx_fldr = r"C:\Users\Laura\Documents\Keepsakes\Travel\0_MetadataInstructions"
xlsx_file = r"Data_Dictionary.xlsx"
xlsx = os.path.join(xlsx_fldr, xlsx_file)

# METADATA VARIABLES
credits = "Schema designed and data populated by Laura Kaufmann (lmmk81914@gmail.com)"
constraints = "Data and schema can only be used with written permission from Laura Kaufmann (lmmk81914@gmail.com)"

# FOLDER OF SQL TXT FILES FOR VIEWS
sqlFldr = r"C:\Users\Laura\Documents\Keepsakes\Travel\0_MetadataInstructions\ViewSQL"

# FUNCTIONS
def getValue(argument):
    if argument == 'NONE':
        return None
    else:
        return argument
    
logging.info('Packages imported; ready to begin')

2023-11-07 20:43:53,305 - Packages imported; ready to begin


In [14]:
# DELETE AND CREATE THE TARGET GEODATABASE
arcpy.management.Delete(os.path.join(fldr, gdb), '')
arcpy.management.CreateFileGDB(fldr, name, "CURRENT")

wrkspc = os.path.join(fldr, gdb)
arcpy.env.overwriteOutput = True

logging.info('Blank file geodatabase created')

2023-11-07 20:43:59,583 - Blank file geodatabase created


In [15]:
# CREATE DOMAINS AND ADD VALUES
domains = pd.read_excel(xlsx, sheet_name='Domains')
domainValues = pd.read_excel(xlsx, sheet_name='DomainValues')

for index, row in domains.iterrows():
    domain_name = row['Name']
    domain_description = row['Description']
    field_type = row['FieldType']
    domain_type = row['DomainType']
    split_policy = row['SplitPolicy']
    merge_policy = row['MergePolicy']

    ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/create-domain.htm
    arcpy.management.CreateDomain(wrkspc, domain_name, domain_description, field_type, domain_type, split_policy, merge_policy)
        
    for index, row in domainValues.iterrows():
        if row['Name'] == domain_name:
            if domain_type == 'CODED':
                ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/add-coded-value-to-domain.htm
                arcpy.management.AddCodedValueToDomain(wrkspc, domain_name, row['Code'], row['ValueDescription'])
            else:
                ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/set-value-for-range-domain.htm
                arcpy.management.SetValueForRangeDomain(wrkspc, domain_name, row['MinValue'], row['MaxValue'])

    logging.info('%s domain and values added to the geodatabase', domain_name)

C:\Users\Laura\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\Laura\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
2023-11-07 20:44:10,661 - VoteType_CL domain and values added to the geodatabase
2023-11-07 20:44:21,376 - TripStage_CL domain and values added to the geodatabase
2023-11-07 20:44:46,235 - DayofWeek_CL domain and values added to the geodatabase
2023-11-07 20:45:05,798 - Currency_CL domain and values added to the geodatabase
2023-11-07 20:45:31,015 - TicketType_CL domain and values added to the geodatabase
2023-11-07 20:46:23,742 - Hour_CL domain and values added to the geodatabase
2023-11-07 20:46:37,103 - Minute_CL domain and values 

In [16]:
# CREATE TABLES
tables = pd.read_excel(xlsx, sheet_name='Tables')
tables = tables.fillna('NONE')

fields = pd.read_excel(xlsx, sheet_name='Fields')
fields = fields.fillna('NONE')

for index, row in tables.iterrows():
    
    out_name = row['Name']
    geometry_type = row['Geometry']
    has_m = row['HasM']
    has_z = row['HasZ']
    summary = row['TableDefinition']
    
    if row['Module'] == 'None':
        tag = geometry_type.capitalize()
    else:
        tag = "{}, {}".format(row['Module'], geometry_type.capitalize())
        
    if geometry_type == 'TABLE':
        ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/create-table.htm
        arcpy.management.CreateTable(wrkspc, out_name, None, '', '')
        logging.info('%s table created in the geodatabase', out_name)
    else:
        ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/create-feature-class.htm
        arcpy.management.CreateFeatureclass(wrkspc, out_name, geometry_type, None, has_m, has_z, sr)
        arcpy.management.RemoveSpatialIndex(os.path.join(wrkspc, out_name))
        logging.info('%s feature class created in the geodatabase', out_name)
    
    mdDesc = []
    
    for index, row in fields.iterrows():
        if row['Table'] == out_name:
            
            field_name = getValue(row['FieldName'])
            field_type = getValue(row['FieldType'])
            field_precision = getValue(row['Precision'])
            field_scale = getValue(row['Scale'])
            field_length = getValue(row['Length'])
            field_alias = getValue(row['FieldAlias'])
            field_is_nullable = getValue(row['Nullable'])
            field_is_required = getValue(row['Required'])
            field_domain = getValue(row['FieldDomain'])
            field_default = getValue(row['DefaultValue'])
            
            ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/add-field.htm
            arcpy.management.AddField(os.path.join(wrkspc, out_name), field_name, field_type, field_precision, field_scale, field_length, field_alias, field_is_nullable, field_is_required, field_domain)
            
            if field_default != None:
                ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/assign-default-to-field.htm
                arcpy.management.AssignDefaultToField(os.path.join(wrkspc, out_name), field_name, field_default)
                setDesc = " (Default: {})".format(field_default)
            
            if field_domain != None:
                setDesc = " ({})".format(field_domain)
            
            if field_default != None and field_domain != None:
                setDesc = " (Default: {} ({}))".format(field_default, field_domain)
            else:
                setDesc = ""
            
            fieldDesc = "{} ({}) - {}{}".format(field_name, field_type.capitalize(), row['FieldDefinition'], setDesc)
            mdDesc.append(fieldDesc)
            
    logging.info('Fields added to %s', out_name)
    
    ##https://pro.arcgis.com/en/pro-app/latest/arcpy/metadata/metadata-class.htm
    new_md = md.Metadata()
    new_md.title = out_name
    new_md.tags = tag
    new_md.summary = summary
    new_md.description = '\n'.join(mdDesc)
    new_md.credits = credits
    new_md.accessConstraints = constraints
    
    tgt_item_md = md.Metadata(os.path.join(wrkspc, out_name))
    if not tgt_item_md.isReadOnly:
        tgt_item_md.copy(new_md)
        tgt_item_md.save()

C:\Users\Laura\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\Laura\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
2023-11-07 20:50:09,194 - Travelers table created in the geodatabase
2023-11-07 20:50:29,649 - Fields added to Travelers
2023-11-07 20:50:32,240 - Travelers_Contacts table created in the geodatabase
2023-11-07 20:50:44,228 - Fields added to Travelers_Contacts
2023-11-07 20:50:46,904 - Travelers_Facts table created in the geodatabase
2023-11-07 20:50:54,195 - Fields added to Travelers_Facts
2023-11-07 20:50:59,608 - Regions feature class created in the geodatabase
2023-11-07 20:51:36,552 - Fields added to Regions
2023-11-07 20:51:39,27

In [17]:
# RELATIONSHIP CLASSES
relationships = pd.read_excel(xlsx, sheet_name='Relationship Classes')
relationships = relationships.fillna('NONE')

for index, row in relationships.iterrows():
    
    origin_table = os.path.join(wrkspc, getValue(row['OriginTable']))
    destination_table = os.path.join(wrkspc, getValue(row['DestinationTable']))
    out_relationship_class = os.path.join(wrkspc, getValue(row['RelationshipClass']))
    relationship_type = getValue(row['RelationshipType'])
    forward_label = getValue(row['ForwardLabel'])
    backward_label = getValue(row['BackwardLabel'])
    message_direction = getValue(row['MessageDirection'])
    cardinality = getValue(row['Cardinality'])
    attributed = getValue(row['Attributed'])
    origin_primary_key = getValue(row['O_PrimaryKey'])
    origin_foreign_key = getValue(row['O_PrimaryKey'])
    destination_primary_key = getValue(row['D_ForeignKey'])
    destination_foreign_key = getValue(row['D_ForeignKey'])
    
    ##https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/create-relationship-class.htm
    arcpy.management.CreateRelationshipClass(origin_table, destination_table, out_relationship_class, relationship_type, forward_label, backward_label, message_direction, cardinality, attributed, origin_primary_key, origin_foreign_key, destination_primary_key, destination_foreign_key)
    logging.info('Relationship created from %s to %s', row['OriginTable'], row['DestinationTable'])

C:\Users\Laura\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
2023-11-07 20:55:50,639 - Relationship created from Trips to Travelers
2023-11-07 20:55:53,948 - Relationship created from Travelers to Travelers_Facts
2023-11-07 20:55:57,551 - Relationship created from Travelers to Travelers_Contacts
2023-11-07 20:56:01,498 - Relationship created from Trips to WorldHex15000
2023-11-07 20:56:05,048 - Relationship created from Trips to Regions
2023-11-07 20:56:08,442 - Relationship created from Regions_Countries to Regions
2023-11-07 20:56:12,132 - Relationship created from Regions to Regions_Averages
2023-11-07 20:56:16,127 - Relationship created from Regions to Locations
2023-11-07 20:56:19,578 - Relationship created from Locations to Locations_Hours
2023-11-07 20:56:23,363 - Relationship created from Locations to Locations_

# VIEWS
views = pd.read_excel(xlsx, sheet_name='Views')
views

for index, row in views.iterrows():
    viewName = "{}.txt".format(row['ViewName'])
    fileName = os.path.join(sqlFldr, viewName)
    
    # Open and read the file as a single buffer
    file = open(fileName, 'r')
    sqlFile = file.read()
    file.close()
    
    print(fileName)

import sqlite3
from sqlite3 import OperationalError

fileName = r'C:\Users\Laura\Documents\Keepsakes\Travel\0_MetadataInstructions\ViewSQL\ActiveRegions.txt'
    
def executeScriptsFromFile(fileName):
    # Open and read the file as a single buffer
    fd = open(fileName, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        # This will skip and report errors
        # For example, if the tables do not yet exist, this will skip over
        # the DROP TABLE commands
        try:
            c.execute(command)
        except OperationalError, msg:
            print("Command skipped: ", msg)
            
arcpy.management.RegisterWithGeodatabase(
    in_dataset=r"C:\Users\Laura\AppData\Roaming\Esri\ArcGISPro\Favorites\Travel_DEV.sde\Travel_DEV.dbo.Itinerary_Visits",
    in_object_id_field="OBJECTID",
    in_shape_field="Shape",
    in_geometry_type="POINT",
    in_spatial_reference='PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]];-20037700 -30241100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision',
    in_extent=None
)

#https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/make-table-view.htm
#https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/create-database-view.htm